In [2]:
import cv2
import numpy as np
import os

In [3]:
def convert_to_text(image):
    """
    Find center coordinates and dimensions of a white box in an image.

    Args:
        image: Input image (numpy array)

    Returns:
        tuple: Normalized (center_x, center_y, width, height) or None if no box found
    """
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Get image dimensions
    h, w = gray.shape[:2]

    # Threshold the image to get white regions
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour (assuming it's the white box)
    if len(contours) == 0:
        return None

    largest_contour = max(contours, key=cv2.contourArea)

    # Get bounding rectangle
    x, y, width, height = cv2.boundingRect(largest_contour)

    # Calculate center coordinates
    center_x = x + width / 2
    center_y = y + height / 2

    # Normalize values between 0 and 1
    norm_center_x = center_x / w
    norm_center_y = center_y / h
    norm_width = width / w
    norm_height = height / h

    return (norm_center_x, norm_center_y, norm_width, norm_height)


In [6]:
# Get list of all image files in subsample_masks folder
mask_files = os.listdir('edited_images/subsample_masks')

if not os.path.exists('edited_images/subsample_masks_txt'):
    print("Making subsample_masks_txt folder")
    os.makedirs('edited_images/subsample_masks_txt')

# Iterate through each mask image
for mask_file in mask_files:
    # Read the image
    mask_path = os.path.join('edited_images/subsample_masks', mask_file)

    points = convert_to_text(cv2.imread(mask_path))

    txt_file = os.path.join('edited_images/subsample_masks_txt',
                                os.path.splitext(mask_file)[0] + '.txt')

    if points is not None:
        with open(txt_file, 'w') as f:
            f.write(f"0 {' '.join(map(str, points))}")
    else:
        with open(txt_file, 'w') as f:
            pass


Making subsample_masks_txt folder
